# KISS Agent Framework - Interactive Tutorial

**Version:** 0.1.7

This notebook provides an interactive tutorial for the KISS (Keep It Simple, Stupid) Agent Framework.

KISS is a lightweight agent framework that implements a ReAct (Reasoning and Acting) loop for LLM agents.

## Prerequisites

Before running this notebook, ensure you have:
1. Installed the KISS framework: `uv sync --group dev`
2. Set up your API keys as environment variables

**Note:** Some cells require API keys to run (GEMINI_API_KEY, OPENAI_API_KEY, ANTHROPIC_API_KEY, etc.)

## Setup and Environment Check

First, let's check that the KISS framework is properly installed and verify API key availability.

In [1]:
# Check KISS installation and version
import os
import sys

# Add src to path if running from the repo root
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from kiss import __version__
print(f"KISS version: {__version__}")

# Check available API keys
api_keys = {
    'GEMINI_API_KEY': os.environ.get('GEMINI_API_KEY'),
    'OPENAI_API_KEY': os.environ.get('OPENAI_API_KEY'),
    'ANTHROPIC_API_KEY': os.environ.get('ANTHROPIC_API_KEY'),
    'TOGETHER_API_KEY': os.environ.get('TOGETHER_API_KEY'),
    'OPENROUTER_API_KEY': os.environ.get('OPENROUTER_API_KEY'),
}

print("\nAPI Key Status:")
for key, value in api_keys.items():
    status = "✓ Set" if value else "✗ Not set"
    print(f"  {key}: {status}")

KISS version: 0.1.7

API Key Status:
  GEMINI_API_KEY: ✓ Set
  OPENAI_API_KEY: ✓ Set
  ANTHROPIC_API_KEY: ✓ Set
  TOGETHER_API_KEY: ✓ Set
  OPENROUTER_API_KEY: ✓ Set


## 1. Your First Agent in 30 Seconds

Let's create a simple math agent that can evaluate expressions using tools.

KISS uses **native function calling** from the LLM providers. Your Python functions become tools automatically. Type hints become schemas. Docstrings become descriptions.

In [2]:
from kiss.core.kiss_agent import KISSAgent

def calculate(expression: str) -> str:
    """Evaluate a math expression."""
    return str(eval(expression))

agent = KISSAgent(name="Math Buddy")
result = agent.run(
    model_name="gemini-2.5-flash",
    prompt_template="Calculate: {question}",
    arguments={"question": "What is 15% of 847?"},
    tools=[calculate]
)
print(result)  # Expected: 127.05


## role="user" #
Calculate: What is 15% of 847?
Asking gemini-2.5-flash...


## role="model" #

```python
calculate(expression='0.15 * 847')
```
#### Usage Information
  - [Token usage: 296/1048576]
  - [Agent budget usage: $0.0000/$10.00]
  - [Global budget usage: $0.0000/$200.00]
  - [Step 1/100]


## role="user" #
127.05
Asking gemini-2.5-flash...


## role="model" #

```python
finish(result='15% of 847 is 127.05')
```
#### Usage Information
  - [Token usage: 724/1048576]
  - [Agent budget usage: $0.0001/$10.00]
  - [Global budget usage: $0.0001/$200.00]
  - [Step 2/100]


## role="user" #
15% of 847 is 127.05
15% of 847 is 127.05


## 2. Custom Structured Output

Unlike other agentic systems, you do not need to specify the output schema for the agent. Just create a suitable "finish" function with parameters. The parameters could be treated as the top level keys in a json format.

In [3]:
from kiss.core.kiss_agent import KISSAgent

# Define a custom finish function with your desired output structure
def finish(
    sentiment: str,
    confidence: float,
    key_phrases: str,
    summary: str
) -> str:
    """
    Complete the analysis with structured results.
    
    Args:
        sentiment: The overall sentiment ('positive', 'negative', or 'neutral')
        confidence: Confidence score between 0.0 and 1.0
        key_phrases: Comma-separated list of key phrases found in the text
        summary: A brief summary of the analysis
    
    Returns:
        The formatted analysis result
    """
    return f"Sentiment: {sentiment} (confidence: {confidence})\nKey phrases: {key_phrases}\nSummary: {summary}"

# Example: Demonstrate the function structure (no LLM call needed)
print("Custom finish function defined with parameters:")
print(f"  - sentiment (str)")
print(f"  - confidence (float)")
print(f"  - key_phrases (str)")
print(f"  - summary (str)")
print("\nThe agent will automatically use this structure for its output.")

Custom finish function defined with parameters:
  - sentiment (str)
  - confidence (float)
  - key_phrases (str)
  - summary (str)

The agent will automatically use this structure for its output.


## 5. KISS Coding Agent - Multi-Agent Coding System

The KISS Coding Agent is a multi-agent system with orchestration and sub-agents that efficiently breaks down complex coding tasks into manageable sub-tasks.

In [8]:
from kiss.agents.coding_agents import KISSCodingAgent

# Create agent with a name
agent = KISSCodingAgent(name="My Coding Agent")

print("KISSCodingAgent created successfully!")
print("\nKey Features:")
print("  - Multi-Agent Architecture: Orchestrator delegates to executor agents")
print("  - Dynamic GEPA Refinement: Auto-refines prompts on failures")
print("  - Path Access Control: Enforces read/write permissions")
print("  - Docker Support: Optional container execution")
print("\nExample usage (requires API keys):")
print('''
result = agent.run(
    prompt_template="Write, test, and optimize a fibonacci function",
    orchestrator_model_name="claude-sonnet-4-5",
    subtasker_model_name="claude-opus-4-5",
    dynamic_gepa_model_name="claude-haiku-4-5",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir=".",
    max_steps=100,
    trials=3
)
''')

KISSCodingAgent created successfully!

Key Features:
  - Multi-Agent Architecture: Orchestrator delegates to executor agents
  - Dynamic GEPA Refinement: Auto-refines prompts on failures
  - Path Access Control: Enforces read/write permissions
  - Docker Support: Optional container execution

Example usage (requires API keys):

result = agent.run(
    prompt_template="Write, test, and optimize a fibonacci function",
    orchestrator_model_name="claude-sonnet-4-5",
    subtasker_model_name="claude-opus-4-5",
    dynamic_gepa_model_name="claude-haiku-4-5",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir=".",
    max_steps=100,
    trials=3
)



## 6. Claude Coding Agent

The Claude Coding Agent uses the Claude Agent SDK to generate tested Python programs with file system access controls.

In [9]:
from kiss.agents.coding_agents import ClaudeCodingAgent

# Create agent with a name
agent = ClaudeCodingAgent(name="My Coding Agent")

print("ClaudeCodingAgent created successfully!")
print("\nBuilt-in Tools Available:")
print("  - Read, Write, Edit, MultiEdit: File operations")
print("  - Glob, Grep: File search and content search")
print("  - Bash: Shell command execution")
print("  - WebSearch, WebFetch: Web access")
print("\nExample usage (requires ANTHROPIC_API_KEY):")
print('''
result = agent.run(
    model_name="claude-sonnet-4-5",
    prompt_template="Write, test, and optimize a fibonacci function",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")
''')

ClaudeCodingAgent created successfully!

Built-in Tools Available:
  - Read, Write, Edit, MultiEdit: File operations
  - Glob, Grep: File search and content search
  - Bash: Shell command execution
  - WebSearch, WebFetch: Web access

Example usage (requires ANTHROPIC_API_KEY):

result = agent.run(
    model_name="claude-sonnet-4-5",
    prompt_template="Write, test, and optimize a fibonacci function",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")



## 7. Gemini CLI Agent

The Gemini CLI Agent uses the Google ADK (Agent Development Kit) to generate tested Python programs.

In [10]:
from kiss.agents.coding_agents import GeminiCliAgent

# Create agent with a name (must be a valid identifier - use underscores, not hyphens)
agent = GeminiCliAgent(name="my_coding_agent")

print("GeminiCliAgent created successfully!")
print("\nNote: Agent name must be a valid Python identifier (use underscores, not hyphens)")
print("\nExample usage (requires GEMINI_API_KEY):")
print('''
result = agent.run(
    model_name="gemini-3-pro-preview",
    prompt_template="Write a fibonacci function with tests",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")
''')

GeminiCliAgent created successfully!

Note: Agent name must be a valid Python identifier (use underscores, not hyphens)

Example usage (requires GEMINI_API_KEY):

result = agent.run(
    model_name="gemini-3-pro-preview",
    prompt_template="Write a fibonacci function with tests",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")



## 8. OpenAI Codex Agent

The OpenAI Codex Agent uses the OpenAI Agents SDK to generate tested Python programs.

In [11]:
from kiss.agents.coding_agents import OpenAICodexAgent

agent = OpenAICodexAgent(name="My Coding Agent")

print("OpenAICodexAgent created successfully!")
print("\nExample usage (requires OPENAI_API_KEY):")
print('''
result = agent.run(
    model_name="gpt-5.2-codex",
    prompt_template="Write a fibonacci function with tests",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")
''')

OpenAICodexAgent created successfully!

Example usage (requires OPENAI_API_KEY):

result = agent.run(
    model_name="gpt-5.2-codex",
    prompt_template="Write a fibonacci function with tests",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir="."
)
if result:
    print(f"Result: {result}")



## 9. ARVO Vulnerability Detector Agent

The ARVO Vulnerability Detector agent uses the Arvo fuzzing framework to discover security vulnerabilities in C/C++ code.

In [12]:
from kiss.agents.arvo_agent.arvo_agent import find_vulnerability, get_all_arvo_tags

print("ARVO Vulnerability Detector imported successfully!")
print("\nFeatures:")
print("  - Runs in Docker container with Arvo fuzzing framework")
print("  - Analyzes code to create hypotheses for vulnerabilities")
print("  - Generates Python scripts to create test inputs for fuzzing")
print("  - Detects ASAN crashes to identify security vulnerabilities")
print("\nExample usage:")
print('''
# Get available Arvo Docker image tags
tags = get_all_arvo_tags("n132/arvo")

# Find vulnerabilities in a specific Docker image
result = find_vulnerability(
    model_name="gemini-3-pro-preview",
    image_name="n132/arvo:tag-name",
    num_trials=10,
    location="/src"
)

if result:
    print(f"Vulnerability found! POC script: {result}")
else:
    print("No vulnerability found after all trials")
''')

ARVO Vulnerability Detector imported successfully!

Features:
  - Runs in Docker container with Arvo fuzzing framework
  - Analyzes code to create hypotheses for vulnerabilities
  - Generates Python scripts to create test inputs for fuzzing
  - Detects ASAN crashes to identify security vulnerabilities

Example usage:

# Get available Arvo Docker image tags
tags = get_all_arvo_tags("n132/arvo")

# Find vulnerabilities in a specific Docker image
result = find_vulnerability(
    model_name="gemini-3-pro-preview",
    image_name="n132/arvo:tag-name",
    num_trials=10,
    location="/src"
)

if result:
    print(f"Vulnerability found! POC script: {result}")
else:
    print("No vulnerability found after all trials")



## 10. SimpleRAG for Retrieval-Augmented Generation

SimpleRAG provides a lightweight RAG system with in-memory vector storage and similarity search.

**Note**: SimpleRAG requires a model with embedding support. Currently, OpenAI, Together AI, and Gemini models support embeddings. Anthropic models do not provide embedding APIs.

In [13]:
from kiss.rag import SimpleRAG

print("SimpleRAG imported successfully!")
print("\nFeatures:")
print("  - In-memory vector storage")
print("  - Similarity search with cosine or L2 distance")
print("  - Document metadata support")
print("  - Filter functions for queries")
print("\nSupported embedding models:")
print("  - OpenAI: text-embedding-3-small, text-embedding-3-large")
print("  - Google: text-embedding-004")
print("  - Together AI: BAAI/bge-large-en-v1.5, m2-bert-80M-32k-retrieval")

SimpleRAG imported successfully!

Features:
  - In-memory vector storage
  - Similarity search with cosine or L2 distance
  - Document metadata support
  - Filter functions for queries

Supported embedding models:
  - OpenAI: text-embedding-3-small, text-embedding-3-large
  - Google: text-embedding-004
  - Together AI: BAAI/bge-large-en-v1.5, m2-bert-80M-32k-retrieval


In [14]:
# SimpleRAG example (requires OPENAI_API_KEY or GEMINI_API_KEY)
# Uncomment to run with API keys

# Initialize RAG system with a model name that supports embeddings
# rag = SimpleRAG(model_name="gpt-4o", metric="cosine")  # or "l2" for L2 distance

# Add documents
documents = [
    {
        "id": "1",
        "text": "Python is a programming language known for its simplicity.",
        "metadata": {"topic": "programming", "language": "Python"},
    },
    {
        "id": "2",
        "text": "Machine learning uses algorithms to learn from data.",
        "metadata": {"topic": "ML", "field": "AI"},
    },
    {
        "id": "3",
        "text": "Docker containers provide isolated execution environments.",
        "metadata": {"topic": "devops", "tool": "Docker"},
    },
]

print("Sample documents for RAG:")
for doc in documents:
    print(f"  [{doc['id']}] {doc['text'][:50]}... (topic: {doc['metadata'].get('topic')})")

# Uncomment the following to run with API keys:
# rag.add_documents(documents)
# 
# # Query similar documents
# results = rag.query("What is Python?", top_k=2)
# for result in results:
#     print(f"ID: {result['id']}, Score: {result['score']:.4f}")
#     print(f"Text: {result['text']}")
#     print()
# 
# # Query with filter
# def filter_by_topic(doc: dict) -> bool:
#     return doc.get("metadata", {}).get("topic") == "programming"
# 
# filtered_results = rag.query("programming language", top_k=5, filter_fn=filter_by_topic)
# 
# # Get collection statistics
# stats = rag.get_collection_stats()
# print(f"Documents: {stats['num_documents']}, Embedding dim: {stats['embedding_dimension']}")

Sample documents for RAG:
  [1] Python is a programming language known for its sim... (topic: programming)
  [2] Machine learning uses algorithms to learn from dat... (topic: ML)
  [3] Docker containers provide isolated execution envir... (topic: devops)


## 11. Useful Agents

The framework includes pre-built utility agents for common tasks.

### Dynamic GEPA Agent

Refines prompts based on agent trajectory analysis.

In [15]:
from kiss.agents.kiss import dynamic_gepa_agent

print("dynamic_gepa_agent imported successfully!")
print("\nUsage:")
print('''
refined_prompt = dynamic_gepa_agent(
    original_prompt_template="Original prompt...",
    previous_prompt_template="Previous version...",
    agent_trajectory_summary=agent.get_trajectory(),  # Returns JSON string
    model_name="gemini-2.5-flash"
)
''')

dynamic_gepa_agent imported successfully!

Usage:

refined_prompt = dynamic_gepa_agent(
    original_prompt_template="Original prompt...",
    previous_prompt_template="Previous version...",
    agent_trajectory_summary=agent.get_trajectory(),  # Returns JSON string
    model_name="gemini-2.5-flash"
)



### General Bash Agent

Runs bash tasks in a sandboxed Ubuntu environment.

In [16]:
from kiss.agents.kiss import run_bash_task_in_sandboxed_ubuntu_latest

print("run_bash_task_in_sandboxed_ubuntu_latest imported successfully!")
print("\nUsage (requires Docker and API keys):")
print('''
result = run_bash_task_in_sandboxed_ubuntu_latest(
    task="Install and configure nginx",
    model_name="gemini-3-pro-preview"
)
''')

run_bash_task_in_sandboxed_ubuntu_latest imported successfully!

Usage (requires Docker and API keys):

result = run_bash_task_in_sandboxed_ubuntu_latest(
    task="Install and configure nginx",
    model_name="gemini-3-pro-preview"
)



### Simple Coding Agent

Creates a simple coding agent with a test function for validation.

In [17]:
from kiss.agents.kiss import get_run_simple_coding_agent

def test_fn(code: str) -> bool:
    """Test if the generated code is correct."""
    try:
        namespace = {}
        exec(code, namespace)
        func = namespace.get('my_function')
        if not func:
            return False
        # Add your test logic here
        return func(42) == 84  # Example test
    except Exception:
        return False

prompt_template = """
Write a Python function called 'my_function' that doubles its input.
The function should be: def my_function(x): return x * 2
"""

# Test the test function
correct_code = "def my_function(x): return x * 2"
wrong_code = "def my_function(x): return x + 2"

print("Testing the test_fn function:")
print(f"  Correct code (x*2) passes: {test_fn(correct_code)}")
print(f"  Wrong code (x+2) fails: {test_fn(wrong_code)}")

# get_run_simple_coding_agent returns a function that can be used to run the agent
run_simple_coding_agent = get_run_simple_coding_agent(test_fn)

print("\nSimple coding agent created! Usage (requires API keys):")
print('''
result = run_simple_coding_agent(
    prompt_template=prompt_template,
    arguments={},
    model_name="gemini-2.5-flash"
)
print(result)
''')

Testing the test_fn function:
  Correct code (x*2) passes: True
  Wrong code (x+2) fails: False

Simple coding agent created! Usage (requires API keys):

result = run_simple_coding_agent(
    prompt_template=prompt_template,
    arguments={},
    model_name="gemini-2.5-flash"
)
print(result)



## 12. Versioning

The project uses semantic versioning (MAJOR.MINOR.PATCH). The version is defined in `src/kiss/_version.py`.

In [18]:
from kiss import __version__
print(f"KISS version: {__version__}")

KISS version: 0.1.7


## Summary

This notebook demonstrated the key features of the KISS Agent Framework:

1. **Simple Agent Creation**: Create agents with just a few lines of code
2. **Custom Output Formatting**: Define finish functions for structured output
3. **Multi-Agent Orchestration**: Compose agents using regular Python
4. **Self-Improving Agents**: Use Dynamic GEPA for prompt refinement
5. **Agent Evolution**: Optimize agents with Pareto frontier maintenance
6. **Coding Agents**: KISS, Claude, Gemini, and OpenAI coding agents
7. **Security Analysis**: ARVO vulnerability detection
8. **RAG Support**: SimpleRAG for retrieval-augmented generation
9. **Utility Agents**: Pre-built agents for common tasks

### How to Run This Notebook

```bash
# From the kiss directory, run:
uv run jupyter notebook kiss.ipynb

# Or use JupyterLab:
uv run jupyter lab kiss.ipynb
```

### Running as a Script

You can also convert this notebook to a Python script and run it:

```bash
uv run jupyter nbconvert --to script kiss.ipynb
uv run python kiss.py
```